### The class NP

_burton rosenberg_
<br/>
_september 15, 2019_

**Definition** A non-deterministic turing machine, NTM, is an abstract computing device
that can have a multiplicity of computation paths. For a polynomial time NTM, P-time NTM, 
every computation path must terminate in a number of steps bound by a polynomial in the 
input size. Each end state is labelled accept or reject. An NTM accepts if some 
path in the computation tree accepts, and rejects if all paths in the computation tree
rejects.

**Definition** A language L is in NP if there exists a P-time NTM A that decides membership in L. That is, 
<pre>
    for all l in L, A(l) accepts
    for all l not in L, A(l) rejects
</pre>

There is an equivalent definition. 

**Definition** A polynomial verifier V  for a language L is a deterministic polynomial time algorithm A(l,w) such that,
<pre>
    for all l in L, there exists an w such that A(l,w) accepts
    for all l not in L, for all w, A(l,w) rejects
</pre>

**Thereom** These definitions are equivalent. A language L is accepted by some P-time NTM
if and only if it is accepted by some P-time verifier. Hence either definition is used
for an NP language.

The following is a Python programs that demonstrates the equivalence. We take as the
language Hamiltonian graphs restricted to graphs where every vertex is degree 3.

Such a family is nontrivial &mdash; There exists an infinite family of graphs of this
sort. I call them <em>torture wheels</em>. Form a ring of 3k nodes, and then for each 
successive triple, connect them to a single new node. This can be drawn with the nodes 
as the tips of spikes sticking from a circle of nodes.

There are also infinite non-hamiltonian graphs that are of regular vertex degree 3 &mdash;
and pair of disjoint torture wheel graphs is non-hamiltonian. The two graphs can be
connected and remain non-hamiltonian by adding a vertex to one edge of each graph
and an edge between these two vertices.



In [21]:
import random

# graph representation.
#    linked list of linked lists where g[i] is a list of edges from node i
#    assert that any node be in range 0 through len(g)-1.


def hamiltonian_verifier(g,w):
    """
    given a graph and a witness w, verify that g is regular degree 3 and
    hamiltonian. the witness w will be the hamiltonian path
    """

    if not regular_graph_p(g,3):
        print("Not a regular graph")
        return False
    
    visited = [False]*len(g)
    first = True
    for node in w:
        if first:
            first = False
            visited[node] = True
            start_node = node
        else:
            if graph_edge_p(g,v,node) and not visited[node]:
                visited[node] = True
            else:
                return False
        v = node

    if graph_edge_p(g,v,start_node):
        for i in visited:
            if not i: return False
        
    return True

def regular_graph_p(g,d):
    """
    given a graph g, is it regular of degree d
    """
    i = 0
    for e_s in g:
        if len(e_s)!=3: return False
        for v in e_s:
            if v==i: 
                return False # not interested in self loops
            if i not in g[v]:
                return False # incorrect, edge must point back
        i += 1
    return True

def graph_edge_p(g,v,u):
    if u in g[v]:
        return True
    return False




def hamiltonian_ntm(g,rt=None):
    """
    Non-deterministic polytime algorithm for reconginzing if the 
    graph g is hamiltonian
    """
    if not regular_graph_p(g,3):
        print("Not a regular graph")
        return False

    if rt==None:
        rt = RandomTape()
    
    visited = [False]*len(g)
    non_visited = len(g)-1
    node = 0
    visited[node] = True
    while non_visited>0:
        v = rt.branch(g[node])
        if visited[v]:
            return False
        visited[v] = True
        node = v
        non_visited -= 1
    return True
    

class RandomTape:
    
    def __init__(self):
        self.randomtape = None
        self.n = 0

    def branch(self,list_choice):
        if self.randomtape == None:
            return random.choice(list_choice)
        self.n += 1
        return self.randomtape[self.n]
    
    def set_tape(self,tape):
        self.n = 0
        self.randomtape = tape


# reductions
        
def ntm_reduceto_verify(g):
    """
    NTM program for Hamiltionian, using a Verifier
    """
    path = [i for i in range(len(g))]
    random.shuffle(path)
    return hamiltonian_verifier(g,path)


def verify_reduceto_ntm(g,w):
    """
    Verifier program for Hamiltionian, using a NTM
    """   
    rt = RandomTape()
    rt.set_tape(w)
    return hamiltonian_ntm(g,rt)
        

#   test programs


def create_wheel(n_spikes):
    spike = n_spikes*3
    g = []
    for v in range(3*n_spikes):
        if v==0:
            g.append([spike-1,spike,1])
            continue
        if v==(3*n_spikes-1):
            g.append([v-1,spike,0])
            continue
        g.append([v-1,spike,v+1])
        if (v+1)%3==0:
            spike += 1
    v = 0
    for i in range(n_spikes):
        g.append([v,v+1,v+2])
        v += 3
    return g



def test():
    simplex = [[1,2,3],[0,2,3],[0,1,3],[1,2,0]]   
    cats_eye = [[1,3],[0,2,4],[1,3],[0,2,4],[1,3]]
    wheel = create_wheel(3)
    
    print("A simplex is a 3 regular hamiltonian graph")
    assert(hamiltonian_verifier(simplex,[0,3,2,1]))
    print("A terror wheel is a 3 regular hamiltonian graph")
    assert(hamiltonian_verifier(wheel,[0,1,9,2,3,10,4,5,6,7,11,8]))    
    print("The cats eye i not a 3 regular graph")
    assert(not hamiltonian_verifier(cats_eye,[0,4,3,2,1]))

    print("Use the NTM in the form of a verifier")
    assert(verify_reduceto_ntm(simplex,[0,3,2,1]))
    
    print("Use the Verifier in the form of a NTM")
    if not ntm_reduceto_verify(wheel):
        print("guess again!")
    print("done")

test()



A simplex is a 3 regular hamiltonian graph
A terror wheel is a 3 regular hamiltonian graph
The cats eye i not a 3 regular graph
Not a regular graph
Use the NTM in the form of a verifier
Use the Verifier in the form of a NTM
guess again!
done
